In [1]:
import sys
sys.path.append("utils")
import dynamic_backtesting as db
import compare_strategies as cs
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib
from matplotlib.dates import DateFormatter
from matplotlib.dates import YearLocator
plt.style.use('seaborn')
matplotlib.rcParams['axes.prop_cycle'] = matplotlib.cycler(color=["grey", "green", "black", "deepskyblue", "orange", "lime", "purple", "Gold", "brown", "blue", "pink"])
plt.rcParams.update({
   "text.usetex": True,
   "font.family": ['Latin Modern Roman'],
   "xtick.labelsize": 20.0,
   "ytick.labelsize": 20.0,
   "axes.labelsize":  20.0,
   "legend.fontsize": 18.0
})  

In [2]:
assets = {"EEM": "Emerging Markets",
           "IVV": "S&P 500",
           "IEV": "MSCI Europe",
           "IXN": "Global Tech",
           "IYR": "US Real Estate",
           "IXG": "Global Financials",
           "EXI": "Global Industrials",
           "GC=F": "Gold Futures", 
           "BZ=F": "Brent Crude Oil Futures",
           "HYG": "High-Yield Corporate Bonds",
           "TLT": "20+ Year Treasury Bonds"}
tickers = list(assets.keys())
stocks = tickers[:7]
commodities = tickers[7:9]
bonds = tickers[9:]

**Laver funktion til at hente vægte og udregne performance for forskellige tickers og gamma**

In [3]:
def backtesting(tickers,Gamma):
    v_t_ARCH_TC, out_of_sample_ARCH_TC, _,Omega_t_ARCH_TC = db.garch_with_trading_cost(tickers=tickers,
    number_of_out_of_sample_days=4*250, model_type="sGARCH10",tuning_gamma_D=Gamma)
    v_t_GARCH_TC, out_of_sample_GARCH_TC, _,Omega_t_GARCH_TC = db.garch_with_trading_cost(tickers=tickers,
    number_of_out_of_sample_days=4*250, model_type="sGARCH11",tuning_gamma_D=Gamma)
    v_t_GJR_TC, out_of_sample_GJR_TC, _,Omega_t_GJR_TC = db.garch_with_trading_cost(tickers=tickers,
    number_of_out_of_sample_days=4*250, model_type="gjrGARCH11",tuning_gamma_D=Gamma)
    res_ARCH_TC, perf_table_ARCH_TC = cs.performance_table(v_t_ARCH_TC, out_of_sample_ARCH_TC,Omega_t_ARCH_TC,portfolio_value=1e9)
    res_GARCH_TC, perf_table_GARCH_TC = cs.performance_table(v_t_GARCH_TC, out_of_sample_GARCH_TC,Omega_t_GARCH_TC,portfolio_value=1e9)
    res_GJR_TC, perf_table_GJR_TC = cs.performance_table(v_t_GJR_TC, out_of_sample_GJR_TC,Omega_t_GJR_TC,portfolio_value=1e9)
    perf_table_TC=perf_table_ARCH_TC.iloc[0:1,]
    perf_table_TC=perf_table_TC.rename(index={'GARCH':'ARCH(1)'})
    perf_table_TC=perf_table_TC.append(perf_table_GARCH_TC.iloc[0,])
    perf_table_TC=perf_table_TC.rename(index={'GARCH':'GARCH(1,1)'})
    perf_table_TC=perf_table_TC.append(perf_table_GJR_TC.iloc[0,])
    perf_table_TC=perf_table_TC.rename(index={'GARCH':'GJR-GARCH(1,1)'})
    perf_table_TC=perf_table_TC.append(perf_table_GJR_TC.iloc[2,])
    perf_table_TC=perf_table_TC.append(perf_table_GJR_TC.iloc[1,])
    perf_table_TC=perf_table_TC.rename(index={'BnH':'Buy and hold','Equal_weight':'Equal weight'})
    #Trading cost
    perf_table_TC=perf_table_TC.append(perf_table_ARCH_TC.iloc[3,])
    perf_table_TC=perf_table_TC.rename(index={'GARCH TC':'ARCH(1) TC'})
    perf_table_TC=perf_table_TC.append(perf_table_GARCH_TC.iloc[3,])
    perf_table_TC=perf_table_TC.rename(index={'GARCH TC':'GARCH(1,1) TC'})
    perf_table_TC=perf_table_TC.append(perf_table_GJR_TC.iloc[3,])
    perf_table_TC=perf_table_TC.rename(index={'GARCH TC':'GJR-GARCH(1,1) TC'})
    perf_table_TC=perf_table_TC.append(perf_table_GJR_TC.iloc[4,])
    perf_table_TC=perf_table_TC.append(perf_table_GJR_TC.iloc[5,])
    perf_table_TC=perf_table_TC.rename(index={'BnH TC':'Buy and hold TC','Equal_weight TC':'Equal weight TC'})

    #gross and net return
    res_tc=res_ARCH_TC.drop(columns=['GARCH TC','Equal_weight', 'BnH','Equal_weight TC','BnH TC']).rename(columns={'GARCH':'ARCH(1)'})
    res_tc["GARCH(1,1)"]=res_GARCH_TC["GARCH"]
    res_tc["GJR-GARCH(1,1)"]=res_GJR_TC["GARCH"]
    res_tc["Equal weight"]=res_GJR_TC["Equal_weight"]
    res_tc["Buy and hold"]=res_GJR_TC["BnH"]

    res_tc_TC=res_ARCH_TC.drop(columns=['GARCH','Equal_weight', 'BnH','Equal_weight TC','BnH TC']).rename(columns={'GARCH TC':'ARCH(1)'})
    res_tc_TC["GARCH(1,1)"]=res_GARCH_TC["GARCH TC"]
    res_tc_TC["GJR-GARCH(1,1)"]=res_GJR_TC["GARCH TC"]
    res_tc_TC["Equal weight"]=res_GJR_TC["Equal_weight TC"]
    res_tc_TC["Buy and hold"]=res_GJR_TC["BnH TC"]
    return v_t_ARCH_TC,v_t_GARCH_TC, v_t_GJR_TC, perf_table_TC,res_tc,res_tc_TC


In [4]:
gamma_low=1.526988705495546e-06
gamma_medium=1.3e-5
gamma_high=1e-4

**All assets, calibrated gamma**

In [ ]:
v_t_ARCH_all_low,v_t_GARCH_all_low, v_t_GJR_all_low, perf_table_all_low,res_all_low,res_all_low_TC = backtesting(tickers,gamma_low)

**Figurer**

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(9,7), dpi=300)
ax.plot(res_all_low,linewidth=1)
ax.set_ylim(0,1.8)
ax.set_xlim("2017-10-08","2021-10-02")
ax.xaxis.set_major_locator(YearLocator())
ax.xaxis.set_major_formatter(DateFormatter("%Y"))
ax.legend([r'\textbf{ARCH(1)}',r'\textbf{GARCH(1,1)}',r'\textbf{GJR-GARCH(1,1)}',r'\textbf{Equal weight}',r'\textbf{Buy and hold}']
,ncol=2,loc="upper left",fontsize=14)
ax.set_ylabel(r'\textbf{Gross Return}',loc='top', rotation = 0, labelpad = -165) 
fig.savefig("Performance_TC_before_L.png", bbox_inches = 'tight', pad_inches = 0.1)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(9,7), dpi=300)
ax.plot(res_all_low_TC,linewidth=1)
ax.set_ylim(0,1.6)
ax.set_xlim("2017-10-08","2021-10-02")
ax.legend([r'\textbf{ARCH(1)}',r'\textbf{GARCH(1,1)}',r'\textbf{GJR-GARCH(1,1)}',r'\textbf{Equal weight}',r'\textbf{Buy and hold}']
,ncol=2,loc="upper left",fontsize=14)
ax.xaxis.set_major_locator(YearLocator())
ax.xaxis.set_major_formatter(DateFormatter("%Y"))
ax.set_ylabel(r'\textbf{Net Return}',loc='top', rotation = 0, labelpad = -145)
fig.savefig("Performance_TC_after_L.png", bbox_inches = 'tight', pad_inches = 0.1)

In [ ]:
v_t_ARCH_all_low.index = pd.to_datetime(v_t_ARCH_all_low.index)
fig, ax = plt.subplots(1, 1, figsize=(9,7), dpi=300)
ax.plot(v_t_ARCH_all_low,linewidth=0.8)
ax.set_ylim(-0.15,1.3)
ax.set_xlim("2017-10-08","2021-10-02")
ax.legend([r"\textbf{Emerging Markets}",r"\textbf{S\&P 500}",r"\textbf{MSCI Europe}",r"\textbf{Global Tech}",r"\textbf{US Real Estate}"
,r"\textbf{Global Financials}",r"\textbf{Global Industrials}",r"\textbf{Gold Futures}",r"\textbf{Brent Crude Oil Futures}"
,r"\textbf{High-Yield Corporate Bonds}",r"\textbf{20+ Year Treasury Bonds}"],ncol=3,loc="upper left",fontsize=10.8)
ax.xaxis.set_major_locator(YearLocator())
ax.xaxis.set_major_formatter(DateFormatter("%Y"))
ax.set_ylabel(r'\textbf{Weight, $v_t$}',loc='top', rotation = 0, labelpad = -128)
fig.savefig("ARCH_weight_TC_L.png", bbox_inches = 'tight', pad_inches = 0.1)

In [ ]:
v_t_GARCH_all_low.index = pd.to_datetime(v_t_GARCH_all_low.index)
fig, ax = plt.subplots(1, 1, figsize=(9,7), dpi=300)
ax.plot(v_t_GARCH_all_low,linewidth=0.8)
ax.set_ylim(-0.15,1.3)
ax.set_xlim("2017-10-08","2021-10-02")
ax.legend([r"\textbf{Emerging Markets}",r"\textbf{S\&P 500}",r"\textbf{MSCI Europe}",r"\textbf{Global Tech}",r"\textbf{US Real Estate}"
,r"\textbf{Global Financials}",r"\textbf{Global Industrials}",r"\textbf{Gold Futures}",r"\textbf{Brent Crude Oil Futures}"
,r"\textbf{High-Yield Corporate Bonds}",r"\textbf{20+ Year Treasury Bonds}"],ncol=3,loc="upper left",fontsize=10.8)
ax.xaxis.set_major_locator(YearLocator())
ax.xaxis.set_major_formatter(DateFormatter("%Y"))
ax.set_ylabel(r'\textbf{Weight, $v_t$}',loc='top', rotation = 0, labelpad = -128)
fig.savefig("GARCH_weight_TC_L.png", bbox_inches = 'tight', pad_inches = 0.1)

In [ ]:
v_t_GJR_all_low.index = pd.to_datetime(v_t_GJR_all_low.index)
fig, ax = plt.subplots(1, 1, figsize=(9,7), dpi=300)
ax.plot(v_t_GJR_all_low,linewidth=0.8)
ax.set_ylim(-0.15,1.3)
ax.set_xlim("2017-10-08","2021-10-02")
ax.legend([r"\textbf{Emerging Markets}",r"\textbf{S\&P 500}",r"\textbf{MSCI Europe}",r"\textbf{Global Tech}",r"\textbf{US Real Estate}"
,r"\textbf{Global Financials}",r"\textbf{Global Industrials}",r"\textbf{Gold Futures}",r"\textbf{Brent Crude Oil Futures}"
,r"\textbf{High-Yield Corporate Bonds}",r"\textbf{20+ Year Treasury Bonds}"],ncol=3,loc="upper left",fontsize=10.8)
ax.xaxis.set_major_locator(YearLocator())
ax.xaxis.set_major_formatter(DateFormatter("%Y"))
ax.set_ylabel(r'\textbf{Weight, $v_t$}',loc='top', rotation = 0, labelpad = -128)
fig.savefig("GJRGARCH_weight_TC_L.png", bbox_inches = 'tight', pad_inches = 0.1)

In [ ]:
print(perf_table_all_low.round(4).to_latex())

**All assets, lowest std**

In [ ]:
v_t_ARCH_all_std,v_t_GARCH_all_std, v_t_GJR_all_std, perf_table_all_std,res_all_std,res_all_std_TC = backtesting(tickers,gamma_medium)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(9,7), dpi=300)
ax.plot(res_all_std,linewidth=1)
ax.set_ylim(0,1.8)
ax.set_xlim("2017-10-08","2021-10-02")
ax.xaxis.set_major_locator(YearLocator())
ax.xaxis.set_major_formatter(DateFormatter("%Y"))
ax.legend([r'\textbf{ARCH(1)}',r'\textbf{GARCH(1,1)}',r'\textbf{GJR-GARCH(1,1)}',r'\textbf{Equal weight}',r'\textbf{Buy and hold}']
,ncol=2,loc="upper left",fontsize=14)
ax.set_ylabel(r'\textbf{Gross Return}',loc='top', rotation = 0, labelpad = -165) 
fig.savefig("Performance_TC_before_M.png", bbox_inches = 'tight', pad_inches = 0.1)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(9,7), dpi=300)
ax.plot(res_all_std_TC,linewidth=1)
ax.set_ylim(0,1.6)
ax.set_xlim("2017-10-08","2021-10-02")
ax.legend([r'\textbf{ARCH(1)}',r'\textbf{GARCH(1,1)}',r'\textbf{GJR-GARCH(1,1)}',r'\textbf{Equal weight}',r'\textbf{Buy and hold}']
,ncol=2,loc="upper left",fontsize=14)
ax.xaxis.set_major_locator(YearLocator())
ax.xaxis.set_major_formatter(DateFormatter("%Y"))
ax.set_ylabel(r'\textbf{Net Return}',loc='top', rotation = 0, labelpad = -145)
fig.savefig("Performance_TC_after_M.png", bbox_inches = 'tight', pad_inches = 0.1)

In [ ]:
v_t_ARCH_all_std.index = pd.to_datetime(v_t_ARCH_all_std.index)
fig, ax = plt.subplots(1, 1, figsize=(9,7), dpi=300)
ax.plot(v_t_ARCH_all_std,linewidth=0.8)
ax.set_ylim(-0.15,1.3)
ax.set_xlim("2017-10-08","2021-10-02")
ax.legend([r"\textbf{Emerging Markets}",r"\textbf{S\&P 500}",r"\textbf{MSCI Europe}",r"\textbf{Global Tech}",r"\textbf{US Real Estate}"
,r"\textbf{Global Financials}",r"\textbf{Global Industrials}",r"\textbf{Gold Futures}",r"\textbf{Brent Crude Oil Futures}"
,r"\textbf{High-Yield Corporate Bonds}",r"\textbf{20+ Year Treasury Bonds}"],ncol=3,loc="upper left",fontsize=10.8)
ax.xaxis.set_major_locator(YearLocator())
ax.xaxis.set_major_formatter(DateFormatter("%Y"))
ax.set_ylabel(r'\textbf{Weight, $v_t$}',loc='top', rotation = 0, labelpad = -128)
fig.savefig("ARCH_weight_TC_M.png", bbox_inches = 'tight', pad_inches = 0.1)

In [ ]:
v_t_GARCH_all_std.index = pd.to_datetime(v_t_GARCH_all_std.index)
fig, ax = plt.subplots(1, 1, figsize=(9,7), dpi=300)
ax.plot(v_t_GARCH_all_std,linewidth=0.8)
ax.set_ylim(-0.15,1.3)
ax.set_xlim("2017-10-08","2021-10-02")
ax.legend([r"\textbf{Emerging Markets}",r"\textbf{S\&P 500}",r"\textbf{MSCI Europe}",r"\textbf{Global Tech}",r"\textbf{US Real Estate}"
,r"\textbf{Global Financials}",r"\textbf{Global Industrials}",r"\textbf{Gold Futures}",r"\textbf{Brent Crude Oil Futures}"
,r"\textbf{High-Yield Corporate Bonds}",r"\textbf{20+ Year Treasury Bonds}"],ncol=3,loc="upper left",fontsize=10.8)
ax.xaxis.set_major_locator(YearLocator())
ax.xaxis.set_major_formatter(DateFormatter("%Y"))
ax.set_ylabel(r'\textbf{Weight, $v_t$}',loc='top', rotation = 0, labelpad = -128)
fig.savefig("GARCH_weight_TC_M.png", bbox_inches = 'tight', pad_inches = 0.1)

In [ ]:
v_t_GJR_all_std.index = pd.to_datetime(v_t_GJR_all_std.index)
fig, ax = plt.subplots(1, 1, figsize=(9,7), dpi=300)
ax.plot(v_t_GJR_all_std,linewidth=0.8)
ax.set_ylim(-0.15,1.3)
ax.set_xlim("2017-10-08","2021-10-02")
ax.legend([r"\textbf{Emerging Markets}",r"\textbf{S\&P 500}",r"\textbf{MSCI Europe}",r"\textbf{Global Tech}",r"\textbf{US Real Estate}"
,r"\textbf{Global Financials}",r"\textbf{Global Industrials}",r"\textbf{Gold Futures}",r"\textbf{Brent Crude Oil Futures}"
,r"\textbf{High-Yield Corporate Bonds}",r"\textbf{20+ Year Treasury Bonds}"],ncol=3,loc="upper left",fontsize=10.8)
ax.xaxis.set_major_locator(YearLocator())
ax.xaxis.set_major_formatter(DateFormatter("%Y"))
ax.set_ylabel(r'\textbf{Weight, $v_t$}',loc='top', rotation = 0, labelpad = -128) 
fig.savefig("GJRGARCH_weight_TC_M.png", bbox_inches = 'tight', pad_inches = 0.1)

In [ ]:
print(perf_table_all_std.round(4).to_latex()) 

**All assets, combination between return and risk**

In [ ]:
v_t_ARCH_all_best,v_t_GARCH_all_best, v_t_GJR_all_best, perf_table_all_best,res_all_best,res_all_best_TC = backtesting(tickers,gamma_high)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(9,7), dpi=300)
ax.plot(res_all_best,linewidth=1)
ax.set_ylim(0,1.8)
ax.set_xlim("2017-10-08","2021-10-02")
ax.xaxis.set_major_locator(YearLocator())
ax.xaxis.set_major_formatter(DateFormatter("%Y"))
ax.legend([r'\textbf{ARCH(1)}',r'\textbf{GARCH(1,1)}',r'\textbf{GJR-GARCH(1,1)}',r'\textbf{Equal weight}',r'\textbf{Buy and hold}']
,ncol=2,loc="upper left",fontsize=14)
ax.set_ylabel(r'\textbf{Gross Return}',loc='top', rotation = 0, labelpad = -165) 
fig.savefig("Performance_TC_before_H.png", bbox_inches = 'tight', pad_inches = 0.1)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(9,7), dpi=300)
ax.plot(res_all_best_TC,linewidth=1)
ax.set_ylim(0,1.6)
ax.set_xlim("2017-10-08","2021-10-02")
ax.legend([r'\textbf{ARCH(1)}',r'\textbf{GARCH(1,1)}',r'\textbf{GJR-GARCH(1,1)}',r'\textbf{Equal weight}',r'\textbf{Buy and hold}']
,ncol=2,loc="upper left",fontsize=14)
ax.xaxis.set_major_locator(YearLocator())
ax.xaxis.set_major_formatter(DateFormatter("%Y"))
ax.set_ylabel(r'\textbf{Net Return}',loc='top', rotation = 0, labelpad = -145) 
fig.savefig("Performance_TC_after_H.png", bbox_inches = 'tight', pad_inches = 0.1)

In [ ]:
v_t_ARCH_all_best.index = pd.to_datetime(v_t_ARCH_all_best.index)
fig, ax = plt.subplots(1, 1, figsize=(9,7), dpi=300)
ax.plot(v_t_ARCH_all_best,linewidth=0.8)
ax.set_ylim(-0.15,1.3)
ax.set_xlim("2017-10-08","2021-10-02")
ax.legend([r"\textbf{Emerging Markets}",r"\textbf{S\&P 500}",r"\textbf{MSCI Europe}",r"\textbf{Global Tech}",r"\textbf{US Real Estate}"
,r"\textbf{Global Financials}",r"\textbf{Global Industrials}",r"\textbf{Gold Futures}",r"\textbf{Brent Crude Oil Futures}"
,r"\textbf{High-Yield Corporate Bonds}",r"\textbf{20+ Year Treasury Bonds}"],ncol=3,loc="upper left",fontsize=10.8)
ax.xaxis.set_major_locator(YearLocator())
ax.xaxis.set_major_formatter(DateFormatter("%Y"))
ax.set_ylabel(r'\textbf{Weight, $v_t$}',loc='top', rotation = 0, labelpad = -128)
fig.savefig("ARCH_weight_TC_H.png", bbox_inches = 'tight', pad_inches = 0.1)

In [ ]:
v_t_GARCH_all_best.index = pd.to_datetime(v_t_GARCH_all_best.index)
fig, ax = plt.subplots(1, 1, figsize=(9,7), dpi=300)
ax.plot(v_t_GARCH_all_best,linewidth=0.8)
ax.set_ylim(-0.15,1.3)
ax.set_xlim("2017-10-08","2021-10-02")
ax.legend([r"\textbf{Emerging Markets}",r"\textbf{S\&P 500}",r"\textbf{MSCI Europe}",r"\textbf{Global Tech}",r"\textbf{US Real Estate}"
,r"\textbf{Global Financials}",r"\textbf{Global Industrials}",r"\textbf{Gold Futures}",r"\textbf{Brent Crude Oil Futures}"
,r"\textbf{High-Yield Corporate Bonds}",r"\textbf{20+ Year Treasury Bonds}"],ncol=3,loc="upper left",fontsize=10.8)
ax.xaxis.set_major_locator(YearLocator())
ax.xaxis.set_major_formatter(DateFormatter("%Y"))
ax.set_ylabel(r'\textbf{Weight, $v_t$}',loc='top', rotation = 0, labelpad = -128)
fig.savefig("GARCH_weight_TC_H.png", bbox_inches = 'tight', pad_inches = 0.1)

In [ ]:
v_t_GJR_all_best.index = pd.to_datetime(v_t_GJR_all_best.index)
fig, ax = plt.subplots(1, 1, figsize=(9,7), dpi=300)
ax.plot(v_t_GJR_all_best,linewidth=0.8)
ax.set_ylim(-0.15,1.3)
ax.set_xlim("2017-10-08","2021-10-02")
ax.legend([r"\textbf{Emerging Markets}",r"\textbf{S\&P 500}",r"\textbf{MSCI Europe}",r"\textbf{Global Tech}",r"\textbf{US Real Estate}"
,r"\textbf{Global Financials}",r"\textbf{Global Industrials}",r"\textbf{Gold Futures}",r"\textbf{Brent Crude Oil Futures}"
,r"\textbf{High-Yield Corporate Bonds}",r"\textbf{20+ Year Treasury Bonds}"],ncol=3,loc="upper left",fontsize=10.8)
ax.xaxis.set_major_locator(YearLocator())
ax.xaxis.set_major_formatter(DateFormatter("%Y"))
ax.set_ylabel(r'\textbf{Weight, $v_t$}',loc='top', rotation = 0, labelpad = -128) 
fig.savefig("GJRGARCH_weight_TC_H.png", bbox_inches = 'tight', pad_inches = 0.1)

In [ ]:
print(perf_table_all_best.round(4).to_latex())  

**Commundities, calibrated gamma**

In [ ]:
v_t_ARCH_com_low,v_t_GARCH_com_low, v_t_GJR_com_low, perf_table_com_low,res_com_low,res_com_low_TC = backtesting(commodities,gamma_low)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(9,7), dpi=300)
ax.plot(res_com_low,linewidth=1)
ax.set_ylim(0,1.8)
ax.set_xlim("2017-10-08","2021-10-02")
ax.xaxis.set_major_locator(YearLocator())
ax.xaxis.set_major_formatter(DateFormatter("%Y"))
ax.legend([r'\textbf{ARCH(1)}',r'\textbf{GARCH(1,1)}',r'\textbf{GJR-GARCH(1,1)}',r'\textbf{Equal weight}',r'\textbf{Buy and hold}']
,ncol=2,loc="upper left",fontsize=14)
ax.set_ylabel(r'\textbf{Gross Return}',loc='top', rotation = 0, labelpad = -165) 
fig.savefig("Performance_TC_com_before_L.png", bbox_inches = 'tight', pad_inches = 0.1)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(9,7), dpi=300)
ax.plot(res_com_low_TC,linewidth=1)
ax.set_ylim(0,1.6)
ax.set_xlim("2017-10-08","2021-10-02")
ax.legend([r'\textbf{ARCH(1)}',r'\textbf{GARCH(1,1)}',r'\textbf{GJR-GARCH(1,1)}',r'\textbf{Equal weight}',r'\textbf{Buy and hold}']
,ncol=2,loc="upper left",fontsize=14)
ax.xaxis.set_major_locator(YearLocator())
ax.xaxis.set_major_formatter(DateFormatter("%Y"))
ax.set_ylabel(r'\textbf{Net Return}',loc='top', rotation = 0, labelpad = -145) 
fig.savefig("Performance_TC_com_after_L.png", bbox_inches = 'tight', pad_inches = 0.1)

In [ ]:
v_t_ARCH_com_low.index = pd.to_datetime(v_t_ARCH_com_low.index)
fig, ax = plt.subplots(1, 1, figsize=(9,7), dpi=300)
ax.plot(v_t_ARCH_com_low,linewidth=0.8)
ax.set_ylim(-0.15,1.3)
ax.set_xlim("2017-10-08","2021-10-02")
ax.legend([r"\textbf{Gold Futures}",r"\textbf{Brent Crude Oil Futures}"],ncol=2,loc="upper left",fontsize=12)
ax.xaxis.set_major_locator(YearLocator())
ax.xaxis.set_major_formatter(DateFormatter("%Y"))
ax.set_ylabel(r'\textbf{Weight, $v_t$}',loc='top', rotation = 0, labelpad = -128)
fig.savefig("ARCH_weight_TC_com_L.png", bbox_inches = 'tight', pad_inches = 0.1)

In [ ]:
v_t_GARCH_com_low.index = pd.to_datetime(v_t_GARCH_com_low.index)
fig, ax = plt.subplots(1, 1, figsize=(9,7), dpi=300)
ax.plot(v_t_GARCH_com_low,linewidth=0.8)
ax.set_ylim(-0.15,1.3)
ax.set_xlim("2017-10-08","2021-10-02")
ax.legend([r"\textbf{Gold Futures}",r"\textbf{Brent Crude Oil Futures}"],ncol=2,loc="upper left",fontsize=12)
ax.xaxis.set_major_locator(YearLocator())
ax.xaxis.set_major_formatter(DateFormatter("%Y"))
ax.set_ylabel(r'\textbf{Weight, $v_t$}',loc='top', rotation = 0, labelpad = -128) 
fig.savefig("GARCH_weight_TC_com_L.png", bbox_inches = 'tight', pad_inches = 0.1)

In [ ]:
v_t_GJR_com_low.index = pd.to_datetime(v_t_GJR_com_low.index)
fig, ax = plt.subplots(1, 1, figsize=(9,7), dpi=300)
ax.plot(v_t_GJR_com_low,linewidth=0.8)
ax.set_ylim(-0.15,1.3)
ax.set_xlim("2017-10-08","2021-10-02")
ax.legend([r"\textbf{Gold Futures}",r"\textbf{Brent Crude Oil Futures}"],ncol=2,loc="upper left",fontsize=12)
ax.xaxis.set_major_locator(YearLocator())
ax.xaxis.set_major_formatter(DateFormatter("%Y"))
ax.set_ylabel(r'\textbf{Weight, $v_t$}',loc='top', rotation = 0, labelpad = -128) 
fig.savefig("GJRGARCH_weight_TC_com_L.png", bbox_inches = 'tight', pad_inches = 0.1)

In [ ]:
print(perf_table_com_low.round(4).to_latex())  

**Commundities, medium gamma**

In [ ]:
v_t_ARCH_com_LSTD,v_t_GARCH_com_LSTD, v_t_GJR_com_LSTD, perf_table_com_LSTD,res_com_LSTD,res_com_LSTD_TC = backtesting(commodities,gamma_medium)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(9,7), dpi=300)
ax.plot(res_com_LSTD,linewidth=1)
ax.set_ylim(0,1.8)
ax.set_xlim("2017-10-08","2021-10-02")
ax.xaxis.set_major_locator(YearLocator())
ax.xaxis.set_major_formatter(DateFormatter("%Y"))
ax.legend([r'\textbf{ARCH(1)}',r'\textbf{GARCH(1,1)}',r'\textbf{GJR-GARCH(1,1)}',r'\textbf{Equal weight}',r'\textbf{Buy and hold}']
,ncol=2,loc="upper left",fontsize=14)
ax.set_ylabel(r'\textbf{Gross Return}',loc='top', rotation = 0, labelpad = -165) 
fig.savefig("Performance_TC_com_before_M.png", bbox_inches = 'tight', pad_inches = 0.1)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(9,7), dpi=300)
ax.plot(res_com_LSTD_TC,linewidth=1)
ax.set_ylim(0,1.6)
ax.set_xlim("2017-10-08","2021-10-02")
ax.legend([r'\textbf{ARCH(1)}',r'\textbf{GARCH(1,1)}',r'\textbf{GJR-GARCH(1,1)}',r'\textbf{Equal weight}',r'\textbf{Buy and hold}']
,ncol=2,loc="upper left",fontsize=14)
ax.xaxis.set_major_locator(YearLocator())
ax.xaxis.set_major_formatter(DateFormatter("%Y"))
ax.set_ylabel(r'\textbf{Net Return}',loc='top', rotation = 0, labelpad = -145) 
fig.savefig("Performance_TC_com_after_M.png", bbox_inches = 'tight', pad_inches = 0.1)

In [ ]:
v_t_ARCH_com_LSTD.index = pd.to_datetime(v_t_ARCH_com_LSTD.index)
fig, ax = plt.subplots(1, 1, figsize=(9,7), dpi=300)
ax.plot(v_t_ARCH_com_LSTD,linewidth=0.8)
ax.set_ylim(-0.15,1.3)
ax.set_xlim("2017-10-08","2021-10-02")
ax.legend([r"\textbf{Gold Futures}",r"\textbf{Brent Crude Oil Futures}"],ncol=2,loc="upper left",fontsize=12)
ax.xaxis.set_major_locator(YearLocator())
ax.xaxis.set_major_formatter(DateFormatter("%Y"))
ax.set_ylabel(r'\textbf{Weight, $v_t$}',loc='top', rotation = 0, labelpad = -128)
fig.savefig("ARCH_weight_TC_com_M.png", bbox_inches = 'tight', pad_inches = 0.1)

In [ ]:
v_t_GARCH_com_LSTD.index = pd.to_datetime(v_t_GARCH_com_LSTD.index)
fig, ax = plt.subplots(1, 1, figsize=(9,7), dpi=300)
ax.plot(v_t_GARCH_com_LSTD,linewidth=0.8)
ax.set_ylim(-0.15,1.3)
ax.set_xlim("2017-10-08","2021-10-02")
ax.legend([r"\textbf{Gold Futures}",r"\textbf{Brent Crude Oil Futures}"],ncol=2,loc="upper left",fontsize=12)
ax.xaxis.set_major_locator(YearLocator())
ax.xaxis.set_major_formatter(DateFormatter("%Y"))
ax.set_ylabel(r'\textbf{Weight, $v_t$}',loc='top', rotation = 0, labelpad = -128)
fig.savefig("GARCH_weight_TC_com_M.png", bbox_inches = 'tight', pad_inches = 0.1)

In [ ]:
v_t_GJR_com_LSTD.index = pd.to_datetime(v_t_GJR_com_LSTD.index)
fig, ax = plt.subplots(1, 1, figsize=(9,7), dpi=300)
ax.plot(v_t_GJR_com_LSTD,linewidth=0.8)
ax.set_ylim(-0.15,1.3)
ax.set_xlim("2017-10-08","2021-10-02")
ax.legend([r"\textbf{Gold Futures}",r"\textbf{Brent Crude Oil Futures}"],ncol=2,loc="upper left",fontsize=12)
ax.xaxis.set_major_locator(YearLocator())
ax.xaxis.set_major_formatter(DateFormatter("%Y"))
ax.set_ylabel(r'\textbf{Weight, $v_t$}',loc='top', rotation = 0, labelpad = -128) 
fig.savefig("GJRGARCH_weight_TC_com_M.png", bbox_inches = 'tight', pad_inches = 0.1)

In [ ]:
print(perf_table_com_LSTD.round(4).to_latex())  

**Commundities, high gamma**

In [ ]:
v_t_ARCH_com_HSR,v_t_GARCH_com_HSR, v_t_GJR_com_HSR, perf_table_com_HSR,res_com_HSR,res_com_HSR_TC = backtesting(commodities,gamma_high)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(9,7), dpi=300)
ax.plot(res_com_HSR,linewidth=1)
ax.set_ylim(0,1.8)
ax.set_xlim("2017-10-08","2021-10-02")
ax.xaxis.set_major_locator(YearLocator())
ax.xaxis.set_major_formatter(DateFormatter("%Y"))
ax.legend([r'\textbf{ARCH(1)}',r'\textbf{GARCH(1,1)}',r'\textbf{GJR-GARCH(1,1)}',r'\textbf{Equal weight}',r'\textbf{Buy and hold}']
,ncol=2,loc="upper left",fontsize=14)
ax.set_ylabel(r'\textbf{Gross Return}',loc='top', rotation = 0, labelpad = -165) 
fig.savefig("Performance_TC_com_before_H.png", bbox_inches = 'tight', pad_inches = 0.1)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(9,7), dpi=300)
ax.plot(res_com_HSR_TC,linewidth=1)
ax.set_ylim(0,1.6)
ax.set_xlim("2017-10-08","2021-10-02")
ax.legend([r'\textbf{ARCH(1)}',r'\textbf{GARCH(1,1)}',r'\textbf{GJR-GARCH(1,1)}',r'\textbf{Equal weight}',r'\textbf{Buy and hold}']
,ncol=2,loc="upper left",fontsize=14)
ax.xaxis.set_major_locator(YearLocator())
ax.xaxis.set_major_formatter(DateFormatter("%Y"))
ax.set_ylabel(r'\textbf{Net Return}',loc='top', rotation = 0, labelpad = -145) 
fig.savefig("Performance_TC_com_after_H.png", bbox_inches = 'tight', pad_inches = 0.1)

In [ ]:
v_t_ARCH_com_HSR.index = pd.to_datetime(v_t_ARCH_com_HSR.index)
fig, ax = plt.subplots(1, 1, figsize=(9,7), dpi=300)
ax.plot(v_t_ARCH_com_HSR,linewidth=0.8)
ax.set_ylim(-0.15,1.3)
ax.set_xlim("2017-10-08","2021-10-02")
ax.legend([r"\textbf{Gold Futures}",r"\textbf{Brent Crude Oil Futures}"],ncol=2,loc="upper left",fontsize=12)
ax.xaxis.set_major_locator(YearLocator())
ax.xaxis.set_major_formatter(DateFormatter("%Y"))
ax.set_ylabel(r'\textbf{Weight, $v_t$}',loc='top', rotation = 0, labelpad = -128)
fig.savefig("ARCH_weight_TC_com_H.png", bbox_inches = 'tight', pad_inches = 0.1)

In [ ]:
v_t_GARCH_com_HSR.index = pd.to_datetime(v_t_GARCH_com_HSR.index)
fig, ax = plt.subplots(1, 1, figsize=(9,7), dpi=300)
ax.plot(v_t_GARCH_com_HSR,linewidth=0.8)
ax.set_ylim(-0.15,1.3)
ax.set_xlim("2017-10-08","2021-10-02")
ax.legend([r"\textbf{Gold Futures}",r"\textbf{Brent Crude Oil Futures}"],ncol=2,loc="upper left",fontsize=12)
ax.xaxis.set_major_locator(YearLocator())
ax.xaxis.set_major_formatter(DateFormatter("%Y"))
ax.set_ylabel(r'\textbf{Weight, $v_t$}',loc='top', rotation = 0, labelpad = -128)
fig.savefig("GARCH_weight_TC_com_H.png", bbox_inches = 'tight', pad_inches = 0.1)

In [ ]:
v_t_GJR_com_HSR.index = pd.to_datetime(v_t_GJR_com_HSR.index)
fig, ax = plt.subplots(1, 1, figsize=(9,7), dpi=300)
ax.plot(v_t_GJR_com_HSR,linewidth=0.8)
ax.set_ylim(-0.15,1.3)
ax.set_xlim("2017-10-08","2021-10-02")
ax.legend([r"\textbf{Gold Futures}",r"\textbf{Brent Crude Oil Futures}"],ncol=2,loc="upper left",fontsize=12)
ax.xaxis.set_major_locator(YearLocator())
ax.xaxis.set_major_formatter(DateFormatter("%Y"))
ax.set_ylabel(r'\textbf{Weight, $v_t$}',loc='top', rotation = 0, labelpad = -128) 
fig.savefig("GJRGARCH_weight_TC_com_H.png", bbox_inches = 'tight', pad_inches = 0.1)

In [ ]:
print(perf_table_com_HSR.round(4).to_latex())  

**Remaning tickers**

In [11]:
_,_, v_t_GJR_bonds_low, perf_table_bonds_low,res_bonds_low,res_bonds_low_TC = backtesting(bonds,gamma_low)

['HYG', 'TLT']
Solving problem with trading costs. gamma_D = 1.526988705495546e-06, tuning gamma_D = 1.526988705495546e-06
['HYG', 'TLT']
Solving problem with trading costs. gamma_D = 1.526988705495546e-06, tuning gamma_D = 1.526988705495546e-06
['HYG', 'TLT']
Solving problem with trading costs. gamma_D = 1.526988705495546e-06, tuning gamma_D = 1.526988705495546e-06
Low portfolio weights sum detected of 0.0006187342681639868


In [12]:
print(perf_table_bonds_low.round(4).to_latex())

\begin{tabular}{lrrr}
\toprule
{} &  Ann. standard deviation &  Ann. return &  Ann. Sharpe ratio \\
\midrule
ARCH(1)           &                   0.0807 &       0.0649 &             0.8047 \\
GARCH(1,1)        &                   0.0703 &       0.0508 &             0.7224 \\
GJR-GARCH(1,1)    &                   0.0744 &       0.0396 &             0.5323 \\
Buy and hold      &                   0.0673 &       0.0516 &             0.7672 \\
Equal weight      &                   0.0756 &       0.0590 &             0.7806 \\
ARCH(1) TC        &                   0.3830 &      -0.5652 &            -1.4759 \\
GARCH(1,1) TC     &                   0.8090 &      -1.0000 &            -1.2361 \\
GJR-GARCH(1,1) TC &                   0.3935 &      -0.4584 &            -1.1647 \\
Equal weight TC   &                   0.0776 &       0.0469 &             0.6043 \\
Buy and hold TC   &                   0.0673 &       0.0516 &             0.7672 \\
\bottomrule
\end{tabular}



In [7]:
_,_, v_t_GJR_bonds_medium, perf_table_bonds_medium,res_bonds_medium,res_bonds_medium_TC = backtesting(bonds, 0.00013)

['HYG', 'TLT']
Solving problem with trading costs. gamma_D = 1.526988705495546e-06, tuning gamma_D = 0.00013
['HYG', 'TLT']
Solving problem with trading costs. gamma_D = 1.526988705495546e-06, tuning gamma_D = 0.00013
['HYG', 'TLT']
Solving problem with trading costs. gamma_D = 1.526988705495546e-06, tuning gamma_D = 0.00013
Low portfolio weights sum detected of 0.0006187342681639868


In [8]:
print(perf_table_bonds_medium.round(4).to_latex())

\begin{tabular}{lrrr}
\toprule
{} &  Ann. standard deviation &  Ann. return &  Ann. Sharpe ratio \\
\midrule
ARCH(1)           &                   0.0658 &       0.0506 &             0.7692 \\
GARCH(1,1)        &                   0.0647 &       0.0501 &             0.7739 \\
GJR-GARCH(1,1)    &                   0.0639 &       0.0530 &             0.8288 \\
Buy and hold      &                   0.0673 &       0.0516 &             0.7672 \\
Equal weight      &                   0.0756 &       0.0590 &             0.7806 \\
ARCH(1) TC        &                   0.0658 &       0.0499 &             0.7584 \\
GARCH(1,1) TC     &                   0.0647 &       0.0489 &             0.7552 \\
GJR-GARCH(1,1) TC &                   0.0639 &       0.0520 &             0.8137 \\
Equal weight TC   &                   0.0776 &       0.0469 &             0.6043 \\
Buy and hold TC   &                   0.0673 &       0.0516 &             0.7672 \\
\bottomrule
\end{tabular}



In [9]:
_,_,v_t_GJR_bonds_high, perf_table_bonds_high,res_bonds_high,res_bonds_high_TC= backtesting(bonds, 0.000213)

['HYG', 'TLT']
Solving problem with trading costs. gamma_D = 1.526988705495546e-06, tuning gamma_D = 0.000213
['HYG', 'TLT']
Solving problem with trading costs. gamma_D = 1.526988705495546e-06, tuning gamma_D = 0.000213
['HYG', 'TLT']
Solving problem with trading costs. gamma_D = 1.526988705495546e-06, tuning gamma_D = 0.000213
Low portfolio weights sum detected of 0.0006187342681639868


In [10]:
print(perf_table_bonds_high.round(4).to_latex())

\begin{tabular}{lrrr}
\toprule
{} &  Ann. standard deviation &  Ann. return &  Ann. Sharpe ratio \\
\midrule
ARCH(1)           &                   0.0661 &       0.0511 &             0.7727 \\
GARCH(1,1)        &                   0.0649 &       0.0503 &             0.7754 \\
GJR-GARCH(1,1)    &                   0.0642 &       0.0518 &             0.8066 \\
Buy and hold      &                   0.0673 &       0.0516 &             0.7672 \\
Equal weight      &                   0.0756 &       0.0590 &             0.7806 \\
ARCH(1) TC        &                   0.0661 &       0.0508 &             0.7685 \\
GARCH(1,1) TC     &                   0.0649 &       0.0499 &             0.7688 \\
GJR-GARCH(1,1) TC &                   0.0642 &       0.0514 &             0.8008 \\
Equal weight TC   &                   0.0776 &       0.0469 &             0.6043 \\
Buy and hold TC   &                   0.0673 &       0.0516 &             0.7672 \\
\bottomrule
\end{tabular}



**Stocks**

In [ ]:
_,_, _, perf_table_stocks_low,_,_ = backtesting(stocks,gamma_low)

In [ ]:
print(perf_table_stocks_low.round(4).to_latex())

In [ ]:
_,_, _, perf_table_stocks_medium,_,_ = backtesting(stocks,0.000103)

In [ ]:
print(perf_table_stocks_medium.round(4).to_latex())

In [ ]:
_,_, _, perf_table_stocks_high,_,_ = backtesting(stocks,1.03e-3)

In [ ]:
print(perf_table_stocks_high.round(4).to_latex())

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(9,7), dpi=300)
ax.plot(res_bonds_medium_TC,linewidth=1)
ax.set_ylim(0,1.6)
ax.set_xlim("2017-10-08","2021-10-02")
ax.legend([r'\textbf{ARCH(1)}',r'\textbf{GARCH(1,1)}',r'\textbf{GJR-GARCH(1,1)}',r'\textbf{Equal weight}',r'\textbf{Buy and hold}']
,ncol=2,loc="upper left",fontsize=14)
ax.xaxis.set_major_locator(YearLocator())
ax.xaxis.set_major_formatter(DateFormatter("%Y"))
ax.set_ylabel(r'\textbf{Net Return}',loc='top', rotation = 0, labelpad = -145) 

In [ ]:
v_t_GJR_bonds_low.index = pd.to_datetime(v_t_GJR_bonds_low.index)
fig, ax = plt.subplots(1, 1, figsize=(9,7), dpi=300)
ax.plot(v_t_GJR_bonds_low,linewidth=0.8)
#ax.set_ylim(-0.15,1.3)
ax.set_xlim("2017-10-08","2021-10-02")
ax.xaxis.set_major_locator(YearLocator())
ax.xaxis.set_major_formatter(DateFormatter("%Y"))
ax.set_ylabel(r'\textbf{Weight, $v_t$}',loc='top', rotation = 0, labelpad = -128)